In [78]:
import pandas as pd
import arrow

In [100]:
URL_ESTACIONES = "https://ssl.smn.gob.ar/dpd/zipopendata.php?dato=estaciones"
URL_TEMPERATURAS = "https://ssl.smn.gob.ar/dpd/zipopendata.php?dato=regtemp"

URL_OUTPUT_TEMP_MAX = "temperaturas-maximas.csv"
URL_OUTPUT_TEMP_MIN = "temperaturas-minimas.csv"

## Toma estaciones y temperaturas y mergea

In [85]:
def get_estaciones():
    estaciones = pd.read_fwf(URL_ESTACIONES, compression="zip", encoding="latin1", skiprows=[1], dtype={"FECHA": str})
    try:
        estaciones_backup = pd.read_csv("estaciones.csv", encoding="utf8")
        estaciones_total = pd.concat([estaciones, estaciones_backup]).drop_duplicates()
        estaciones_total.to_csv("estaciones.csv", encoding="utf8", index=False)
    except:
        estaciones_total = estaciones
    return estaciones_total

In [86]:
estaciones = get_estaciones()
estaciones.head()

,NOMBRE,PROVINCIA,LATITUD,LONGITUD,ALTURA,NRO,NroOACI
0,BASE BELGRANO II,ANTARTIDA,-77 52,-34 34,256.0,89034.0,SAYB
1,BASE CARLINI (EX JUBANY),ANTARTIDA,-62 14,-58 38,11.0,89053.0,SAYJ
2,BASE ESPERANZA,ANTARTIDA,-63 24,-56 59,24.0,88963.0,SAYE
3,BASE MARAMBIO,ANTARTIDA,-64 14,-56 43,198.0,89055.0,SAWB
4,BASE ORCADAS,ANTARTIDA,-60 45,-44 43,12.0,88968.0,SAYO


In [87]:
estaciones_dict = {row[1]["NroOACI"]: row[1]["NOMBRE"] for row in estaciones[["NOMBRE", "NroOACI"]].iterrows()}

In [88]:
def get_temperaturas():
    temperaturas = pd.read_fwf(URL_TEMPERATURAS, compression="zip", encoding="latin1", skiprows=[2], header=1, dtype={"FECHA": str})
    try:
        temperaturas_backup = pd.read_csv("temperaturas.csv", encoding="utf8")
        temperaturas_total = pd.concat([temperaturas, temperaturas_backup]).drop_duplicates()
        temperaturas_total.to_csv("temperaturas.csv", encoding="utf8", index=False)
    except:
        temperaturas_total = temperaturas
    return temperaturas_total

In [90]:
temperaturas = get_temperaturas()
temperaturas.head()

,FECHA,TMAX,TMIN,NOMBRE
0,14112019,30.2,20.3,AEROPARQUE AERO
1,14112019,28.5,13.4,AZUL AERO
2,14112019,27.1,13.0,BAHIA BLANCA AERO
3,14112019,18.8,2.5,BARILOCHE AERO
4,14112019,-2.2,-10.0,BASE BELGRANO II


In [91]:
def get_temperaturas_estaciones(temperaturas, estaciones):
    temperaturas_estaciones = temperaturas.merge(estaciones[["NOMBRE", "NroOACI"]], on="NOMBRE")[["FECHA", "TMAX", "TMIN", "NroOACI"]]
    temperaturas_estaciones["FECHA"] = temperaturas_estaciones["FECHA"].apply(lambda x: arrow.get(x, "DDMMYYYY").format("YYYY-MM-DD"))
    return temperaturas_estaciones

In [92]:
temperaturas_estaciones = get_temperaturas_estaciones(temperaturas, estaciones)
temperaturas_estaciones.head()

,FECHA,TMAX,TMIN,NroOACI
0,2019-11-14,30.2,20.3,SABE
1,2019-11-13,27.0,20.4,SABE
2,2019-11-12,25.0,18.8,SABE
3,2019-11-11,23.1,17.8,SABE
4,2019-11-10,22.4,17.5,SABE


## Convierte paneles en series

In [93]:
def rename_columns(col, prefix):
    if col == "FECHA":
        return "indice_tiempo"
    else:
        return "{}_{}".format(prefix.lower(), col.lower())

In [94]:
def temperatures_panel_to_series(df_panel, field_values="TMAX", prefix="temperatura_maxima"):
    df_series = temperaturas_estaciones.pivot_table(index="FECHA", values=field_values, columns="NroOACI").reset_index().sort_values("FECHA")
    df_series.columns = [rename_columns(col, prefix) for col in df_series.columns]
    return df_series

In [95]:
temp_max_series = temperatures_panel_to_series(temperaturas_estaciones, "TMAX", "temperatura_maxima")
temp_min_series = temperatures_panel_to_series(temperaturas_estaciones, "TMIN", "temperatura_minima")

In [96]:
temp_min_series.head()

,indice_tiempo,temperatura_minima_saac,temperatura_minima_saag,temperatura_minima_saai,temperatura_minima_saaj,temperatura_minima_saap,temperatura_minima_saar,temperatura_minima_saav,temperatura_minima_saba,temperatura_minima_sabe,...,temperatura_minima_sazm,temperatura_minima_sazn,temperatura_minima_sazp,temperatura_minima_sazq,temperatura_minima_sazr,temperatura_minima_sazs,temperatura_minima_sazt,temperatura_minima_sazv,temperatura_minima_sazx,temperatura_minima_sazy
0,2018-11-15,13.3,14.2,15.0,13.0,16.0,13.0,14.0,18.0,19.5,...,10.9,12.3,14.2,14.0,12.6,1.9,8.5,12.0,13.4,3.8
1,2018-11-16,14.6,15.5,9.6,9.5,17.3,16.6,18.3,11.4,12.3,...,3.5,5.3,7.1,7.0,7.1,-3.6,-0.3,5.0,8.7,0.4
2,2018-11-17,18.8,16.2,15.0,15.0,18.3,20.0,20.0,17.8,18.3,...,11.8,12.0,13.0,10.8,10.5,4.4,10.0,10.0,13.9,4.2
3,2018-11-18,16.0,13.2,11.5,11.8,13.6,12.4,13.5,13.2,15.0,...,9.9,10.2,9.4,7.6,6.1,5.5,7.3,9.0,9.7,4.2
4,2018-11-19,9.5,11.8,13.0,9.2,11.4,10.1,13.1,15.8,16.0,...,9.3,8.2,11.4,10.4,11.6,7.1,7.5,11.0,12.2,7.3


In [101]:
temp_max_series.to_csv(URL_OUTPUT_TEMP_MAX, encoding="utf8", index=False)
temp_min_series.to_csv(URL_OUTPUT_TEMP_MIN, encoding="utf8", index=False)

## Genera metadatos

In [97]:
def get_metadata_from_columns(columns, title_prefix, description_prefix, id_prefix):
    
    def get_description_from_title(title):
        if title == "indice_tiempo":
            return "Indice de tiempo."
        else:
            return "{}. Estación {}.".format(
                description_prefix,
                estaciones_dict[title.split("_")[-1].upper()].title()
            )
    
    titles = list(columns)
    descriptions = [get_description_from_title(title) for title in titles]
    ids = [title.replace(title_prefix, id_prefix) for title in titles]
    
    return pd.DataFrame({
        "field_title": titles,
        "field_description": descriptions,
        "field_id": ids
    })

In [98]:
temp_min_metadata = get_metadata_from_columns(temp_min_series.columns, "temperatura_minima", "Temperatura mínima diaria", "temp_min")
temp_min_metadata.to_clipboard()
temp_min_metadata.head()

,field_title,field_description,field_id
0,indice_tiempo,Indice de tiempo.,indice_tiempo
1,temperatura_minima_saac,Temperatura mínima diaria. Estación Concordia ...,temp_min_saac
2,temperatura_minima_saag,Temperatura mínima diaria. Estación Gualeguayc...,temp_min_saag
3,temperatura_minima_saai,Temperatura mínima diaria. Estación Punta Indi...,temp_min_saai
4,temperatura_minima_saaj,Temperatura mínima diaria. Estación Junin Aero.,temp_min_saaj


In [99]:
temp_max_metadata = get_metadata_from_columns(temp_max_series.columns, "temperatura_maxima", "Temperatura máxima diaria", "temp_max")
temp_max_metadata.to_clipboard()
temp_max_metadata.head()

,field_title,field_description,field_id
0,indice_tiempo,Indice de tiempo.,indice_tiempo
1,temperatura_maxima_saac,Temperatura máxima diaria. Estación Concordia ...,temp_max_saac
2,temperatura_maxima_saag,Temperatura máxima diaria. Estación Gualeguayc...,temp_max_saag
3,temperatura_maxima_saai,Temperatura máxima diaria. Estación Punta Indi...,temp_max_saai
4,temperatura_maxima_saaj,Temperatura máxima diaria. Estación Junin Aero.,temp_max_saaj
